In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle

In [2]:
data=pd.read_csv('C:\\Users\\user\\Desktop\\datas\\news.csv')
data

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [3]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
data.tail()

,Unnamed: 0,title,text,label
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL
6334,4330,Jeb Bush Is Suddenly Attacking Trump. Here's W...,Jeb Bush Is Suddenly Attacking Trump. Here's W...,REAL


In [5]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [6]:
data.values

array([[8476, 'You Can Smell Hillary’s Fear',
        'Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI.

In [7]:
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [8]:
data.describe()

,Unnamed: 0
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [9]:
data.info

<bound method DataFrame.info of       Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fell

In [10]:
data[:60]

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [11]:
x=data['text']
y=data['label']

In [12]:
x

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [13]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

## Split data into X & Y

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [15]:
x_train

2402    Christian Whiton is a former deputy special en...
1922    Super Tuesday Brings Harsh Light And Heartache...
3475    Prev post Page 1 of 4 Next \nNurses are among ...
6197    The deadly hostage situation at a luxury hotel...
4748    Our new country: Women and minorities hit hard...
                              ...                        
4931    Twenty-two of the 37 corporations nominated fo...
3264    As pieces of luggage, human remains, wreckage ...
1653    0 Add Comment \nIN THE immediate aftermath of ...
2607    Palestine Palestinians check the flat of Amjad...
2732    For the second week in a row, there was a temp...
Name: text, Length: 5068, dtype: object

In [16]:
x_test

3789    Watch the above reports by CBN's David Brody a...
733     — Bernie Sanders (@BernieSanders) October 27, ...
4783    \nAs of October 29, there have been at least 1...
3067    Members of Congress have said that if Hillary ...
5288    Julian Zelizer is a professor of history and p...
                              ...                        
5121    On the front lines of the battle against the I...
6112    Several gunmen seized a luxury hotel in Mali’s...
2661      Paul Joseph Watson Bureaucrat who said “stup...
59      Republican presidential front-runner Donald Tr...
4573    You can buy snacks, condoms, fishing bait, mar...
Name: text, Length: 1267, dtype: object

In [17]:
y_train

2402    REAL
1922    REAL
3475    FAKE
6197    REAL
4748    FAKE
        ... 
4931    REAL
3264    REAL
1653    FAKE
2607    FAKE
2732    REAL
Name: label, Length: 5068, dtype: object

In [18]:
y_test

3789    REAL
733     FAKE
4783    FAKE
3067    FAKE
5288    REAL
        ... 
5121    REAL
6112    REAL
2661    FAKE
59      REAL
4573    REAL
Name: label, Length: 1267, dtype: object

## Now we need to fit the TFIDF Vectorizer.
# max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
# max_df = 25 means "ignore terms that appear in more than 25 documents".

In [19]:
tfvect=TfidfVectorizer(stop_words='english', max_df=0.7)
tfid_x_train=tfvect.fit_transform(x_train)
tfid_x_test=tfvect.transform(x_test)

In [20]:
print(tfid_x_train)

  (0, 5434)	0.04558113027328849
  (0, 22112)	0.03101556231290688
  (0, 41980)	0.041224221139183156
  (0, 41399)	0.08707226417897605
  (0, 32656)	0.05826505252086948
  (0, 40634)	0.08300352400740964
  (0, 59168)	0.03304164565844389
  (0, 17072)	0.015280054882096362
  (0, 21122)	0.03467645674134631
  (0, 36500)	0.036461647252457555
  (0, 21811)	0.021815222286389203
  (0, 55510)	0.037920004934453785
  (0, 35473)	0.025244871570555313
  (0, 8579)	0.03467645674134631
  (0, 34917)	0.05447353166186797
  (0, 51186)	0.027977956004355133
  (0, 17007)	0.020253495378206955
  (0, 13362)	0.023133406428959745
  (0, 28054)	0.026346675461032608
  (0, 51479)	0.04368398576036871
  (0, 19911)	0.03226033674797147
  (0, 12812)	0.021077575874407357
  (0, 5104)	0.0369438299052932
  (0, 22586)	0.03830998824285092
  (0, 23706)	0.03360195028163217
  :	:
  (5067, 25947)	0.0690536695761875
  (5067, 20495)	0.060909276073445075
  (5067, 46270)	0.05755520504375683
  (5067, 56317)	0.0733883544907381
  (5067, 59168)	0.0

In [21]:
print(tfid_x_test)

  (0, 60873)	0.04199293292033408
  (0, 60756)	0.03102467179929353
  (0, 60501)	0.025828759280643834
  (0, 60332)	0.022016410662716892
  (0, 60328)	0.029457870632407204
  (0, 60325)	0.05310926248634118
  (0, 60321)	0.037769691024057916
  (0, 60309)	0.028599297592098793
  (0, 60242)	0.02078664460607839
  (0, 60238)	0.02304876060617083
  (0, 60112)	0.06661620569458904
  (0, 60053)	0.026671179085014728
  (0, 59998)	0.08675332085991751
  (0, 59769)	0.018014389086953634
  (0, 59489)	0.01857186718038564
  (0, 59463)	0.029849288660946164
  (0, 59407)	0.03540084521652464
  (0, 59298)	0.03597852101133186
  (0, 59294)	0.07902793135900811
  (0, 59273)	0.025874119034496358
  (0, 59218)	0.030026827265587194
  (0, 59168)	0.03570687883899036
  (0, 59054)	0.055031944270905395
  (0, 58993)	0.05992243785134022
  (0, 58912)	0.049809921799913816
  :	:
  (1266, 5288)	0.04448311123524479
  (1266, 4956)	0.017480678533077217
  (1266, 4712)	0.02575856859832166
  (1266, 4225)	0.027560846706771686
  (1266, 3720)	

## Now let's fit the Machine Learning Model

In [22]:
classifier=PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

## Now let's check model accuracy. Let's fit model on the test data.

In [23]:
y_pred=classifier.predict(tfid_x_test)
score=accuracy_score(y_test,y_pred)

In [24]:
print(f'Accuracy:{round(score*100,2)}%')

Accuracy:93.45%


# Accuracy: 93.45%

In [25]:
cf=confusion_matrix(y_test,y_pred,labels=['FAKE','REAL'])
print(cf)

[[572  43]
 [ 40 612]]


# [[572  43]
# [ 40 612]]

## Now, let's check our model is working on the custom news or not, let's build a function for it and we will save the model into a pickle file so later we can use the model in the flask app.

## Let's create function for test the model on the real-time data.

In [26]:
def fake_news_det(news):
    input_data=[news]
    vectorized_input_data=tfvect.transform(input_data)
    prediction=classifier.predict(vectorized_input_data)
    print(prediction)

In [27]:
fake_news_det("""Go to Article President Barack Obama has been 
campaigning hard for the woman who is supposedly going to extend his legacy 
four more years. The only problem with stumping for Hillary Clinton, however, 
is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

['FAKE']


# ['FAKE']

# Let's save the model into pickle file so that we can use it in flask app.

In [28]:
pickle.dump(classifier,open('model.pkl','wb'))